In [1]:
import pandas as pd
import numpy as np
import os
from wav2sleep.data.edf import load_edf_data
from wav2sleep.data.txt import parse_txt_annotations
from wav2sleep.data.utils import interpolate_index
from wav2sleep.data.xml import parse_xml_annotations
from wav2sleep.settings import *
from wav2sleep.config import *


In [2]:
import pandas as pd
from pathlib import Path
from itertools import chain   



dfs = []
for name in MASTER_CSV_LIST.keys():
    dataset = MASTER_CSV_LIST[name]
    for file_path in dataset:
        df_temp = pd.read_csv(file_path)
        df_temp['dataset_name'] = name
        
        # set path and unique_id
        # META_PATH = '/scratch/besp/shared_data'
        df_temp['path'] = ""
        storage_path = 'preprocessed_' + name + '/' + name + '/ingest/'# to be decided
        
        if name == 'shhs':
            ######### as an example for path ################
            # for each row, if row['visit']==1, then path = storage_path + name + 'shhs1' + row['nsrrid'].parquet
            print(df_temp['visitnumber'].unique())
            def build_path(row):
                if row["visitnumber"] == 1:
                    return META_PATH + storage_path + name + "-shhs1-" + f"{row.nsrrid}.parquet"
                if row["visitnumber"] == 2:
                    return META_PATH + storage_path + name + "-shhs2-" + f"{row.nsrrid}.parquet"
                return None         

            df_temp["path"] = df_temp.apply(build_path, axis=1)
            #########################################
            df_temp['nsrrid'] = 'shhs' + str(df_temp['nsrrid'])
            
            pass
        elif name == 'chat':
            df_temp['nsrrid'] = 'chat' + str(df_temp['nsrrid'])
            pass
        elif name == 'mros':
            df_temp['nsrrid'] = 'mros' + str(df_temp['nsrrid'])
            pass
        elif name == 'ccshs':
            df_temp['nsrrid'] = 'ccshs' + str(df_temp['nsrrid'])
            pass
        elif name == 'cfs':
            df_temp['nsrrid'] = 'cfs' + str(df_temp['nsrrid'])
            pass
        elif name == 'mesa':
            df_temp['nsrrid'] = 'mesa' + str(df_temp['mesaid'])
            pass
        elif name == 'sof':
            df_temp['nsrrid'] = 'sof' + str(df_temp['nsrrid'])
            pass
        else:
            raise ValueError("no matching dataset error code: a08rh293bidbg1")
        df_temp.set_index("nsrrid", inplace=True)  
        print(df_temp.shape)
        dfs.append(df_temp)


df = pd.concat(dfs, axis=0)
print(df.shape)


[1 2]
(10115, 30)
(906, 31)
(2911, 20)
(517, 36)
(735, 20)
(2237, 29)
(461, 27)
(17882, 51)


In [3]:

df.dropna(subset=['path'], inplace=True)
print(df.shape)
df['path']

(17882, 51)


nsrrid
shhs0        200001\n1        200002\n2        200003\n3        200004\n4        200005\n          ...  \n10110    205798\n10111    205799\n10112    205800\n10113    205801\n10114    205802\nName: nsrrid, Length: 10115, dtype: int64    /scratch/besp/shared_datapreprocessed_shhs/shh...
shhs0        200001\n1        200002\n2        200003\n3        200004\n4        200005\n          ...  \n10110    205798\n10111    205799\n10112    205800\n10113    205801\n10114    205802\nName: nsrrid, Length: 10115, dtype: int64    /scratch/besp/shared_datapreprocessed_shhs/shh...
shhs0        200001\n1        200002\n2        200003\n3        200004\n4        200005\n          ...  \n10110    205798\n10111    205799\n10112    205800\n10113    205801\n10114    205802\nName: nsrrid, Length: 10115, dtype: int64    /scratch/besp/shared_datapreprocessed_shhs/shh...
shhs0        200001\n1        200002\n2        200003\n3        200004\n4        200005\n          ...  \n10110    205798\n10111    205

In [4]:
df.head()

,visitnumber,nsrr_age,nsrr_age_gt89,nsrr_sex,nsrr_race,nsrr_ethnicity,nsrr_bmi,nsrr_bp_systolic,nsrr_bp_diastolic,nsrr_current_smoker,...,nsrr_cai,nsrr_oai,nsrr_oahi_hp4u,nsrr_oahi_hp3u,nsrr_avglvlsa,nsrr_minlvlsa,rectype,mesaid,examnumber,nsrr_ttldursp_s1s4
nsrrid,,,,,,,,,,,,,,,,,,,,,
"shhs0 200001\n1 200002\n2 200003\n3 200004\n4 200005\n ... \n10110 205798\n10111 205799\n10112 205800\n10113 205801\n10114 205802\nName: nsrrid, Length: 10115, dtype: int64",1.0,55.0,no,male,white,not hispanic or latino,21.777553,143.0,79.0,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"shhs0 200001\n1 200002\n2 200003\n3 200004\n4 200005\n ... \n10110 205798\n10111 205799\n10112 205800\n10113 205801\n10114 205802\nName: nsrrid, Length: 10115, dtype: int64",1.0,78.0,no,male,white,not hispanic or latino,32.950680,168.0,68.0,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"shhs0 200001\n1 200002\n2 200003\n3 200004\n4 200005\n ... \n10110 205798\n10111 205799\n10112 205800\n10113 205801\n10114 205802\nName: nsrrid, Length: 10115, dtype: int64",1.0,77.0,no,female,white,not hispanic or latino,24.114150,127.0,68.0,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"shhs0 200001\n1 200002\n2 200003\n3 200004\n4 200005\n ... \n10110 205798\n10111 205799\n10112 205800\n10113 205801\n10114 205802\nName: nsrrid, Length: 10115, dtype: int64",1.0,48.0,no,male,white,not hispanic or latino,20.185185,130.0,83.0,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"shhs0 200001\n1 200002\n2 200003\n3 200004\n4 200005\n ... \n10110 205798\n10111 205799\n10112 205800\n10113 205801\n10114 205802\nName: nsrrid, Length: 10115, dtype: int64",1.0,66.0,no,female,black or african american,not hispanic or latino,23.309053,145.0,78.0,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
